# Forecast.io Project

## Author:

Ruobin Wu (ruobinwu@yahoo.com)

## Completion Date:

Apr 8, 2017 (Eastern Daylight Time)

## Steps:

1. read in csv file
2. convert postal code to geocode
3. expand the date range, generate a sub-dataframe for each row and combine all sub-dataframes into a large one 
4. convert the large dataframe into pivot table
5. generate daily weather info in csv format and excel report of precipitation

## Code:

### Imported packages

In [1]:
import requests
import pandas as pd
import googlemaps
import csv
import os
import sys
import json
import datetime

### Record class
create a record class, the objects of the class are rows in locations.csv file

In [2]:
class Record(object):
    def __init__(self, record):
        self.locId = record[0]
        self.postalCode = record[1]
        self.startDate = record[2] 
        self.endDate = record[3]
        self.lat = None
        self.lng = None

### Function of reading csv file
create a function to read in the csv file using package csv; convert postal code to geocode using Google Maps API;

In [3]:
def readCSV(fileName):
    f = open(fileName, 'r')
    reader = csv.reader(f)
    records = []
    # skip the header of csv file
    next(reader)
    for row in reader:
        record = Record(row)
        # convert postal code to geocode
        record.lat, record.lng = getGeocode(record.postalCode)        
        records.append(record)
    f.close()
    return(records)

# the API Key is in local environment variable
def getGeocode(postalCode):
    google_maps = googlemaps.Client(key=os.environ['GOOGLE_MAPS_API_KEY'])
    location = google_maps.geocode(postalCode)
    if not location:
        return(None, None)
    geolocation = location[0]['geometry']['location']
    return(geolocation['lat'], geolocation['lng'])

### Function of creating a sub-dataframe

create a sub-dataframe for weather info of a specific location and its day period

In [4]:
def subDataFrame(record):
    dfArray = []
    for time in range(int(record.startDate), int(record.endDate) + 1, 60 * 60 * 24):
        apiKey = os.environ['FORECAST_IO_API_KEY']
        URL = "https://api.darksky.net/forecast/"
        URL += apiKey + "/" + str(record.lat) + "," + str(record.lng) + "," + str(time) + "?"
        URL += "exclude=currently,minutely,hourly,alerts,flags"
        try:
            r = requests.get(URL)
            r.raise_for_status()
        except requests.exceptions.HTTPError as err:
            print(err)
            sys.exit(1)
        rJSON = r.text
        weather = json.loads(rJSON)
        daily = weather['daily']['data'][0]

        # add days and locations to daily weather information
        daily['days'] = datetime.datetime.fromtimestamp(time).strftime('%Y-%m-%d')
        daily['locations'] = record.locId

        for key in daily:
            daily[key] = [daily[key]]

        df = pd.DataFrame(daily)
        dfArray.append(df)

    subDataFrame = pd.concat(dfArray)
    return(subDataFrame)

### Function to write data into Excel file
generate a excel report using data on a pivot table

In [1]:
def writeToExcel(table):
    writer = pd.ExcelWriter('precipitation.xlsx', engine='xlsxwriter')
    table.to_excel(writer, 'Sheet1', na_rep='NaN')
    # add percentage sign to numbers
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    fmt = workbook.add_format({'num_format': '0%'})
    index = 1
    while (index <= len(table.columns)):
        worksheet.set_column(index, index, None, fmt)
        index += 1
    writer.save()

### Main function

In [6]:
if __name__ == "__main__":
    # Step 1 & 2: read csv file and get geocode
    records = readCSV('locations.csv')
    # keep the original order of records
    originalLocations = [r.locId for r in records]
    # Step 3: merge each sub-dataframe to a large dataframe
    dataFrame = []
    for record in records:
        dataFrame.append(subDataFrame(record))
    df = pd.concat(dataFrame)

#### print a sample of first 10 rows of the dataframe

In [7]:
df[0:10]

,apparentTemperatureMax,apparentTemperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,cloudCover,days,dewPoint,humidity,icon,locations,...,sunriseTime,sunsetTime,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,time,visibility,windBearing,windSpeed
0,50.46,1485961200,42.53,1485907200,0.95,2017-01-31,48.09,0.96,fog,9be000ae23275d57e1273d211a54ffd7,...,1485934901,1485967844,50.46,1485961200,46.65,1485907200,1485907200,4.39,173,8.52
0,51.10,1486047600,42.80,1486018800,0.80,2017-02-01,47.03,0.90,partly-cloudy-day,9be000ae23275d57e1273d211a54ffd7,...,1486021207,1486054355,51.10,1486047600,48.01,1486018800,1485993600,6.73,173,14.22
0,44.42,1486126800,31.72,1486159200,0.65,2017-02-02,41.89,0.86,partly-cloudy-day,9be000ae23275d57e1273d211a54ffd7,...,1486107511,1486140865,49.72,1486126800,39.57,1486162800,1486080000,7.20,175,11.07
0,44.77,1486220400,32.39,1486166400,0.23,2017-02-03,36.80,0.89,partly-cloudy-day,9be000ae23275d57e1273d211a54ffd7,...,1486193814,1486227376,47.23,1486220400,35.52,1486188000,1486166400,7.47,188,3.30
0,34.89,1486306800,32.69,1486278000,0.49,2017-02-04,37.55,0.95,partly-cloudy-day,9be000ae23275d57e1273d211a54ffd7,...,1486280114,1486313888,40.84,1486306800,37.03,1486278000,1486252800,5.31,25,3.99
0,39.93,1486389600,31.39,1486350000,0.31,2017-02-05,35.86,0.90,partly-cloudy-night,9be000ae23275d57e1273d211a54ffd7,...,1486366413,1486400399,44.36,1486389600,32.72,1486364400,1486339200,4.85,163,3.81
0,45.70,1486476000,32.85,1486425600,0.52,2017-02-06,39.52,0.89,fog,9be000ae23275d57e1273d211a54ffd7,...,1486452710,1486486911,48.87,1486476000,38.09,1486504800,1486425600,5.12,213,0.72
0,36.71,1486515600,30.18,1486594800,0.89,2017-02-07,35.25,0.90,partly-cloudy-day,9be000ae23275d57e1273d211a54ffd7,...,1486539005,1486573422,39.48,1486515600,35.42,1486594800,1486512000,4.70,22,5.37
0,28.10,1486648800,26.16,1486670400,1.00,2017-02-08,31.23,0.89,fog,9be000ae23275d57e1273d211a54ffd7,...,1486625299,1486659934,35.11,1486648800,33.52,1486681200,1486598400,4.96,50,7.82
0,27.62,1486742400,25.62,1486764000,0.89,2017-02-09,32.14,0.94,fog,9be000ae23275d57e1273d211a54ffd7,...,1486711591,1486746445,34.48,1486742400,32.72,1486749600,1486684800,2.49,23,8.03


In [8]:
    # Step 4: convert data frame into pivot table
    table = pd.pivot_table(df, 
                        values = 'precipProbability', 
                        index = ['locations'],
                        columns = ['days'])
    # reset the index back to the original order
    table = table.reindex(originalLocations)

#### print a sample of the pivot table

In [10]:
table

days,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,2017-02-08,2017-02-09,...,2017-02-19,2017-02-20,2017-02-21,2017-02-22,2017-02-23,2017-02-24,2017-02-25,2017-02-26,2017-02-27,2017-02-28
locations,,,,,,,,,,,,,,,,,,,,,
9be000ae23275d57e1273d211a54ffd7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a35e427b4130be7b2a892e286f0ebb91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.67,0.66,0.0,0.52,0.64,0.57,0.74,0.61
185674a2eb5c14fbdbb1d05a4109ea55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
afbddd7f957a1c822293616e95a2d84c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81cea1e224ad183b751acce139f4e276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5f1ce9b7c8cd32c08d98310540fb6604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.98,0.00,0.97,0.0,0.98,0.97,0.93,0.98,0.98
5180af03094779de849ca816c9f5b753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7f2aa8e72612f9130e06b32a0d2a58d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8b049b660e984912c48da213f2f7c650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
    # Step 5: write daily weather info into csv file and precipitation info into Excel file
    df.to_csv('daily_weather_info.csv', na_rep = 'NaN', index = False)
    writeToExcel(table)

## Insight:
1. Four locations that have precipitation data are St Albans, Patchway, Stratford-upon-Avon, and Reading in UK. 
2. The later three show great consistency in precipitation between Feb 19 to 28, 2017.
3. The precipitation of three locations varied dramically during these days.